## Neural Graph Collaborative Filtering
- Paper Link: https://arxiv.org/pdf/1905.08108v2.pdf
- PyTorch Code: https://github.com/huangtinglin/NGCF-PyTorch
- PyTorch Code2: https://bladejun.tistory.com/67

### Abstract
>Learning vector representations (aka. embeddings) of users and
items lies at the core of modern recommender systems. Ranging
from early matrix factorization to recently emerged deep learning
based methods, existing efforts typically obtain a user’s (or an
item’s) embedding by mapping from pre-existing features that
describe the user (or the item), such as ID and attributes. **We
argue that an inherent drawback of such methods is that, the
collaborative signal, which is latent in user-item interactions,
is not encoded in the embedding process.** As such, the resultant
embeddings may not be sufficient to capture the collaborative
filtering effect.  
In this work, **we propose to integrate the user-item interactions —
more specifically the bipartite graph structure — into the embedding
process. We develop a new recommendation framework Neural
Graph Collaborative Filtering (NGCF), which exploits the useritem graph structure by propagating embeddings on it.** This leads
to the expressive modeling of high-order connectivity in useritem graph, effectively injecting the collaborative signal into the
embedding process in an explicit manner. We conduct extensive
experiments on three public benchmarks, demonstrating significant
improvements over several state-of-the-art models like HOPRec [40] and Collaborative Memory Network [5]. Further analysis
verifies the importance of embedding propagation for learning
better user and item representations, justifying the rationality and
effectiveness of NGCF. Codes are available at https://github.com/
xiangwang1223/neural_graph_collaborative_filtering.

기존의 GNN 기반의 Model들을 살펴보게 되면, 주변 Node들의 관계를 고려하여 Prediction이 가능한 Model들이 제시되었다. 하지만, 이러한 Model들의 단점은 Entitiy가 단 하나였다는 것 이다. 즉, User가 어떠한 Class이냐 혹은 Item이 어떠한 Class인지에 대해서만 Prediction을 진행하였다는 것 이다.

현재, NGCF는 이러한 모델들의 문저점은 **Interaction**을 고려하지 못했다는 것 이다. 해당 논문 저자는 이러한 문제점을 해결하기 위하여, **User-Item Relation**을 고려할 수 있는 Collaborative Filtering을 기존 GNN에 추가한 Model을 제안한다.

### Introduction
>Collaborative filtering (CF) addresses it by assuming that behaviorally similar users would exhibit similar preference
on items. To implement the assumption, a common paradigm is to parameterize users and items for reconstructing historical interactions, and predict user preference based on the parameters [1,
14].  
two key components in learnable CF models — 1) embedding, which transforms users and items to vectorized representations, and 2) interaction modeling, which reconstructs historical interactions based on the embeddings.  
Despite their effectiveness, we argue that these methods are not sufficient to yield satisfactory embeddings for CF. The key reason is that the embedding function lacks an explicit encoding of the crucial
collaborative signal, which is latent in user-item interactions to reveal the behavioral similarity between users (or items).  
In this work, we tackle the challenge by exploiting the high-order connectivity from user-item interactions, a natural way that encodes collaborative signal in the interaction graph structure.

해당 논문의 Introduction에서 주요하다고 생각하는 4 문장이다. 현재, 기존 문제점은 다음과 같다.
1. GNN 기반의 Model은 User-Item간의 Interaction을 고려하지 못한다.
2. CF 기반의 Model은 User-Item간의 Interaction을 고려하지만, GNN이 고려할 수 있는 User-User Relation을 고려하지 못한다.
3. 이러한 문제점을 해결하기 위해서는 각각의 문제점을 해결하여야 한다.
    - (1) CF기반의 Model을 만들기 위해서는 User, Item을 각각 Embedding을 할 수 있어야 한다. 또한, 이 두 Vector를 활용하여, User-Item간의 Intercation을 나타낼 수 있어야 한다.
    - (2) User-User Intercation과 같은 GNN의 기반의 Model로서 학습하기 위해서는, High-Order Connectivity을 고려할 수 있어야 한다. 즉, Hop을 키우면 키울수록 고려해야 하는 Dataset의 크기는 매우 커지게 된다. 이러한 문제를 해결할 수 있어야 한다. -> **Embedding Propagation으로서 해결 한다.**
    
먼저, High-Order Interaction을 고려해야 하는 이유를 살펴보자.
![png](../img/NGCF/img1.png)

위의 그림을 살펴보게 되면, 먼저 왼쪽 Sub Figure을 User와 Item간의 Interaction을 나타내는 Figure이다. 오른쪽 Sub Figure는 Hop(<span>$l$</span>)을 늘려가면서 관계를 살펴보는 단계이다. 해당 그림에서는 크게 2가지를 알 수 있다.

첫째, User간의 Interaction을 알 수 있다. 예를 들어 Hop=1(<span>$l=1$</span>)인 경우에는 User-User Interaction을 고려할 수 없다. 하지만, Hop=2(<span>$l=2$</span>)인 경우에는 <span>$u_2 \rightarrow i_2 \rightarrow u_1$</span>으로서 <span>$<u_1, u_2>$</span>의 관계가 있다는 것을 알 수 있다. 즉, 같은 Item(<span>$i_2$</span>)을 선호하는 있다는 관계를 고려할 수 있다는 것 이다.

둘째, 알 수 없던 관계를 고려할 수 있다. 예를 들어 Hop=1(<span>$l=1$</span>)인 경우에는 <span>$u_1$</span>과 <span>$l_4$</span>는 전혀 관계가 없다. 하지만 Hop=3(<span>$l=3$</span>)인 경우에는 <span>$<u_1, i_4>$</span>의 관계가 무려 2개로서 제일 많게 변하는 것을 알 수 있다. 이러한 High-Order Connectivity를 고려하면 할 수록, 이전에는 발견할 수 없는 관계를 고려할 수 있으며, 주요한 Interaction을 찾을 수 있다는 것 이다.

### Methodology

![png](../img/NGCF/img2.png)

위의 Figure는 <span>$u_1$</span>과 <span>$i_4$</span>간의 Interaction을 Prediction하는 Figure이다. 해당 그림을 살펴보게 되면, 아래와 같이 3단계로 이루워진다.
- (1) User와 Item을 각각 Embedding한다.
- (2) User와 Item을 각각 Embedding Propagation을 진행한다. 해당 과정에서 Interaction을 고려하게 된다.
- (3) 주변 User와 Item을 고려하여 Interaction을 Prediction한다.

#### Embedding Layer

먼저 Embedding Lookup Table (<span>$E$</span>)는 아래와 같이 선언된다.
<p>$$E=[\underbrace{e_{u_1}, \ldots, e_{u_N}}_{\text{User Embeddings}}, \underbrace{e_{i_1}, \ldots, e_{i_N}}_{\text{Item Embeddings}}].$$</p>

- <span>$e_u \in \mathbb{R}^{d}$</span>
- <span>$e_i \in \mathbb{R}^{d}$</span>

해당 과정에서 주요한 점은 같은 Diemnsion의 크기로서 Embedding을 진행한다는 것 이다.

#### Embedding Propagation Layers
먼저, GNN의 message-passing architecture에서 CF signal(User-Item Interaction)을 고려하기 위하여 어떻게 Architecture를 구성하였는지 알아보자. 아래 수식은 one-layer propagation에 대하여 먼저 알아보고, multiple successive layers에 대하여 알아보자.

#### First-order Propagation
먼저, user's preference와 item간의 direct한 관계를 고려하기 위한 embedding propagation에 대해 알아보자. 해당 과정은 message construction과 message aggregation으로서 이루워진다.

**Message Construction**  
먼저, Adjancy Matrix에서 이어진 <span>$(u, i)$</span>에 대하여 i가 u에 전달하는 message를 아래와 같이 정의하였다.
<p>$$m_{u \leftarrow i} = f(e_i, e_u, p_{ui})$$</p>

- <span>$m_{u \leftarrow i}$</span>: Message Embedding
- <span>$f(\cdot)$</span>: Message encoding function
- <span>$p_{ui}$</span>: Edge(u,i)의 propagation에 대한 decay factor를 조절하기 위한 인자 이다.

Embeding Function (<span>$f(\cdot)$</span>)은 아래와 같이 정의하였다.
<p>$$m_{u \leftarrow i} = \frac{1}{\sqrt{\| N_u \| \| N_i\|}}(W_1 e_i + W_2(e_i \odot e_u))$$</p>

- <span>$W_1, W_2 \in \mathbb{R}^{d' \times d}$</span>: Trainable Weight
- <span>$\odot$</span>: Element-wise product
- <span>$d'$</span>: Transformation Size
- <span>$p_{ui} = \frac{1}{\sqrt{\| N_u \| \| N_i\|}}$</span>: Laplacian Norm
- <span>$N_i, N_u$</span>: First-hop neighbors of user and item


위의 수식을 살펴보게 되면, 주요하게 살펴보아야 할 점은 두 가지 이다.  
**첫째, <span>$W_2(e_i \odot e_u)$</span>의 수식을 통하여 User와 Item간의 Interaction을 고려할 수 있다.**  
**둘째, Laplacian Norm으로서 Normalization을 실시해야 했다는 것 이다.**

Laplacian Norm에 대해 생각하면, 아래 그림과 같다.

![png](https://wikidocs.net/images/page/176711/hop.png)<br>
그림출처: <a href="https://wikidocs.net/176711">wikidocs</a>

위의 그림을 살펴보게 되면, <span>$p_{ui}$</span>의 의미를 알 수 있다.  
**<span>$p_{ui}$</span>는 아이템 i가 유저 u의 선호도에 얼만큼 공헌했는지를 의미한다. 아이템 i를 소비한 유저가 많다면,  그 중 한 유저에게 보네는 영향력이 작아지도록 Normalization을 실시한다는 의미이다. 또한, Message Passing관점에서 살펴보게 되면, Hop이 길어짐에 따라 점차적으로 공헌도가 감소한다는 특징이 있다.**

**Message Aggregation**  
Item과 User간의 Interaction을 나타낼 수 있는 One Layer Propagation을 나타내었다면, 이제 Aggregation을 어떻게 실시하는지 알아보자. Message Aggregation은 아래와 같이 나타낸 다.

<p>$$e_{u}^{(1)} = \text{LeakyReLU}(m_{u \leftarrow u} + \sum_{i \in N_u}m_{u \leftarrow i})$$</p>

위의 수식을 살펴보게 되면, Message Construction에서 살펴본, <span>$m_{u \leftarrow i}$</span>의 합과 <span>$m_{u \leftarrow u}$</span>가 추가적으로 사용되는 것을 알 수 있다. 

<p>$$m_{u \leftarrow u} = W_1 e_u$$</p>

- <span>$W_1$</span>: The weight matrix shared with the one used in message construction

위의 수식에서는 - <span>$m_{u \leftarrow u}$</span>는 Self-Connection으로서 GCN에서 Residual Network와 같은 역할을 한다고 할 수 있다.

**최종적으로 First-Order Propagation을 살펴보면 아래와 같이 정리할 수 있다.**
1. 해당 논문에서 CF Signal (User-Item Interaction)을 잡기 위하여 <span>$W_2(e_i \odot e_u)$</span>을 사용하였다.
2. Normalization을 실시하기 위하여 <span>$p_{ui}$</span>를 사용하였다. 해당 Normalization을 통하여 User-User Relationship을 이어주는 Item의 경우, 많은 사람이 사용할 수록 Contirubtion이 적도록 Normalization을 실시하였다.
3. <span>$m_{u \leftarrow u}$</span>을 통하여 기존의 자기 자신의 정보를 그대로 전파하는 Self-Connection의 구조를 사용하였다.

#### High-order Propagation
![png](../img/NGCF/img3.png)

1차원 Layer가 아닌 여러 Layer를 쌓은 High-Order Propagation의 형태로 일반화 한 수식은 아래와 같다.

<p>$$e_{u}^{(l)} = \text{LeakyReLU}(m_{u \leftarrow u}^{(l)} + \sum_{i \in N_u}m_{u \leftarrow i}^{(l)})$$</p>

- <span>$m_{u \leftarrow i}^{(l)} = \frac{1}{\sqrt{\| N_u \| \| N_i\|}}(W_1^{(l)} e_i^{(l-1)} + W_2^{(l)}(e_i^{(l-1)} \odot e_u^{(l-1)}))$</span>
- <span>$m_{u \leftarrow u} = W_1^{(l)} e_u^{(l-1)}$</span>
- <span>$W_1^{(l)}, W_2^{(l)} \in \mathbb{R}^{d_l \times d_{l-1}}$</span>: Trainable Transform

**Propagation Rule in Matrix Form**  
위의 수식은 하나의 Element에 관한 수식이다. 해당 수식을 Matrix에 적용하여 계산하기 위해서는 추가적으로 수식을 변경해야 한다. 우리가 Coding을 할 최종적인 수식은 아래와 같다.


<p>$$E^{(l)} = \text{LeakyReLU}((L+I)E^{(l-1)}W_1^{(l)} + LE^{(l-1)} \odot E^{(l-1)}W_2^{(l)})$$</p>

- <span>$E^{(l)} \in \mathbb{R}^{(N+M) \times d_l}$</span>: The Representations for users and items.
- <span>$L = D^{-\frac{1}{2}} A D^{-\frac{1}{2}}$</span>: Laplacian Matrix
    - <span>$A = \begin{bmatrix}
                   0 & R \\
                   R^T & 0
                    \end{bmatrix}$</span>
    - <span>$L_{ui} = 1/\sqrt{\| N_u \| \| N_i \|}$</span>
- <span>$R \in \mathbb{R}^{N \times M}$</span>
    - <span>$N$</span>: Number of users
    - <span>$M$</span>: Number of movies

Matrix Propagation으로서 나타내어 매우 복잡해 보이지만, 간단한 수식 입니다. 해당 수식에 대해서는 <a href="https://wikidocs.net/176711">wikidocs</a>에서 매우 자세히 설명되어 있습니다.

해당 Post에 아래 부분은 해당 <a href="https://wikidocs.net/176711">wikidocs</a>의 내용을 가져왔습니다.

Propagation Rule in Matrix Form는 아래와 같이 <span>$L$</span> Matrix에 대해서 이해하면 수식이 쉬워집니다. 해당 Matrix를 구하기 위해서는 크게 아래와 같이 2가지 Step으로서 구할 수 있습니다.

**Step 1. <span>$A, D$</span>를 구한다.**  
![png](https://wikidocs.net/images/page/176711/laplacian1.png)<br>
그림출처: <a href="https://wikidocs.net/176711">wikidocs</a>

**Step 2. <span>$L = D^{-\frac{1}{2}} A D^{-\frac{1}{2}}$</span>를 구한다.**  
![png](https://wikidocs.net/images/page/176711/laplacian2.png)<br>
![png](https://wikidocs.net/images/page/176711/laplacian3.png)<br>
그림출처: <a href="https://wikidocs.net/176711">wikidocs</a>

위의 수식의 의미를 생각하면 <span>$L_{ui} = 1/\sqrt{\| N_u \| \| N_i \|}$</span>을 만족하는 것을 알 수 있다.

위의 <span>$L$</span>를 활용하면 <span>$(L+I)E^{(l-1)}$</span>의 수식의 의미는 아래와 같이 이해할 수 있다.

![png](https://wikidocs.net/images/page/176711/laplacian4.png)<br>
그림출처: <a href="https://wikidocs.net/176711">wikidocs</a>

즉, 최종적인 아래 수식은 Term 2개로서 각각 이해할 수 있다.  
<p>$$E^{(l)} = \text{LeakyReLU}(\underbrace{ (L+I)E^{(l-1)} }_{\text{Term 1}} W_1^{(l)} + \underbrace{ LE^{(l-1)} \odot E^{(l-1)} }_{\text{Term 2}} W_2^{(l)})$$</p>

- Term 1: 자기 자신의 정보 뿐만 아니라 이어져있는 Item Node간의 관계를 고려한 값
- Term 2: User와 Item Interaction을 고려한 값

#### Model Prediction

Model Prediction은 아래와 같이 정의된다.
<p>$$\hat{y}_{\text{NGCF}}(u,i) = e_u^{*T} e_i^*$$</p>

- <span>$e_u^* = e_u^{(0)} \| \ldots \|e_u^{(l)}$</span>
- <span>$e_i^* = e_i^{(0)} \| \ldots \|e_i^{(l)}$</span>
- <span>$l$</span>: Number of layers
- <span>$\| $</span>: Concat

**해당 논문에서 Prediction은 위와 같이 정의하였고, 중요한 점은 모든 Layer를 Concat하여 Prediction에 사용하였다는 것 이다. 이러한 이유에 대하여 해당 논문에서는 아래와 같이 설명하고 있다.**

>Since the representations obtained in different layers emphasize the messages passed over different connections, they have different contributions in reflecting user preference. 

#### Optimization

<p>$$\text{Loss} = \sum_{u,i,j \in O} - \ln \sigma(\hat{y}_{ui} - \hat{y}_{uj}) + \lambda \| \Theta \|_2^2$$</p>

- <span>$O = \{ (u,i,j)|(u,i) \in R^+, (u,j) \in R^- \}$</span>
- <span>$R^+$</span>: Observed Interastions
- <span>$R^-$</span>: Unobserved Interactions
- <span>$\Theta = \{ E, \{ W_1^{(l)}, W_2^{(l)} \}_{l=1}^L \}$</span>: Trainable model parameters


해당 Loss는 BRP Loss라고 불리는 형태 이다. 해당 Loss의 의미는 사용한 Item과 사용하지 않은 Item을 고려할 수 있으며, 특히 사용한 Item을 더 고려한다는 Loss Format이라고 알려져 있다. 자세한 내용은 <a href="https://killerwhale0917.tistory.com/27">killerwhale0917 Blog</a>를 참조하다. (아직, 해당 Loss에 대해서는 자세히 공부하지 않았습니다.)

### Experiment
해당 논문에서는 top-K recommendation and preference ranking으로서 performance를 비교하였다. recall@K, ndcg@K로서 Performance를 측정하였으며, K=20으로 설정하였다.

#### Dataset
해당 논문에서 사용한 Dataset은 3개이며, 각각은 아래와 같다.

![png](../img/NGCF/img4.png)